In [1]:
import json
from redditqa.data import huggingface_dataset
from redditqa.data import qa_generation

import importlib
importlib.reload(huggingface_dataset)
importlib.reload(qa_generation)

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


<module 'redditqa.data.qa_generation' from '/workspaces/reddit_qa/redditqa/data/qa_generation.py'>

In [2]:
dataset = huggingface_dataset.load_reddit_dataset('/scratch1/redditqa/data/ask_historians/AskHistorians.jsonl')
dataset

Saving the dataset (2/2 shards): 100%|██████████| 116452/116452 [00:03<00:00, 37478.70 examples/s]


Dataset({
    features: ['question_created_utc', 'question_retrieved_on', 'question_deleted', 'question_title', 'question_selftext', 'question_score', 'question_char_length', 'question_selftext_char_length', 'answers'],
    num_rows: 116452
})

In [3]:
dataset = dataset.shuffle(seed=42)

In [9]:
train_pct = 0.6
eval_pct = 0.2
test_pct = 0.2

train_end_index = int(len(dataset) * train_pct)
eval_end_index = train_end_index + int(len(dataset) * eval_pct)
print(f"Train ends at {train_end_index}. Eval ends at {eval_end_index}")

dataset_train = dataset.select(range(0, train_end_index))
dataset_eval = dataset.select(range(train_end_index, eval_end_index))
dataset_test = dataset.select(range(eval_end_index, len(dataset)))

print(f"Train size: {len(dataset_train)}")
print(f"Eval size: {len(dataset_eval)}")
print(f"Test size: {len(dataset_test)}")

Train ends at 69871. Eval ends at 93161
Train size: 69871
Eval size: 23290
Test size: 23291


In [14]:
selection_dict = {
    "train": dataset_train["question_title"],
    "eval": dataset_eval["question_title"],
    "test": dataset_test["question_title"]
}

with open("train_test_dev_questions.json", "w") as f:
    json.dump(selection_dict, f, indent=4)